<a href="https://colab.research.google.com/github/radiohead-lgl/demo/blob/master/Traffic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount('/content/drive/') 

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
# 放弃两个数据集合并的想法,先考虑单个数据集的算法情况
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# 设置显示所有列
pd.set_option('display.max_columns',None)

In [5]:
# 交通事故数据
ori_US_all = pd.read_csv(r'/content/drive/MyDrive/Colab Notebooks/MyTraffic/US_Accidents_Dec21.csv')
ori_US_all.drop(columns={'Number','End_Lat','End_Lng','Wind_Chill(F)','End_Time','Humidity(%)'},inplace=True)

In [6]:
import datetime

In [7]:
ori_US_all['Start_Time'] = pd.to_datetime(ori_US_all['Start_Time'])
ori_US_all['Year'] = ori_US_all['Start_Time'].dt.year
ori_US_all['Month']=ori_US_all['Start_Time'].dt.month
ori_US_all['Weekday']=ori_US_all['Start_Time'].dt.weekday
ori_US_all['Hour']=ori_US_all['Start_Time'].dt.hour
ori_US_all['Day']=ori_US_all['Start_Time'].dt.day
ori_US_all['minutes'] = ori_US_all['Start_Time'].dt.minute + ori_US_all['Hour']*60

In [8]:
# 按年月日分组
# 分组后,对每天的数据进行计算Haversine distance 和 time
ori_US_all['YMD'] = ori_US_all['Start_Time'].dt.strftime('%Y-%m-%d')

In [9]:
US_day = ori_US_all[['ID','Start_Time','Start_Lat','Start_Lng','Year','Month','Hour','Day','minutes','YMD']]

In [10]:
US_day_2016 = US_day[US_day['Year']==2016]


In [11]:
# YMD_US_2016 = US_day_2016.groupby(by='YMD')
YMD_US = US_day.groupby(by='YMD')

In [12]:
from  geopy.distance  import  geodesic 
def pointdesc(lat1,lng1,lat2,lng2):
    desc = geodesic((lat1,lng1), (lat2,lng2)).m
    return desc

In [13]:
# 当时间之差小于15,距离小于250时,认为是重复数据
def every_YMD_func(tmp):
    i = 0
    for begin_index,begin_row in tmp.iterrows():
        i+=1
        # begin_minutes = begin_row['minutes']
        for end_index,end_row in tmp.iloc[i:,:].iterrows():
            if (abs(end_row['minutes']-begin_row['minutes'])<=15):
                # print(abs(end_row['minutes']-begin_row['minutes']),begin_index,end_index)
                if pointdesc(begin_row['Start_Lat'],begin_row['Start_Lng'],end_row['Start_Lat'],end_row['Start_Lng'])<250 :
                    duplicate_list.append(begin_index)
                    break;
# duplicate_list
# tmp.iloc[:,:]

In [ ]:
duplicate_list =[]
for i in YMD_US.groups.keys():
    tmp = YMD_US.get_group(i)
    every_YMD_func(tmp)
duplicate_list

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/MyTraffic/duplicate_index.csv','w',newline='')as csvfile:
  writer = csv.writer(csvfile)
  for row in duplicate_list:
    writer.writerow(row)

In [4]:
f = open('/content/drive/MyDrive/Colab Notebooks/MyTraffic/duplicate_index.txt','w')
f.write(duplicate_list)
f.close()

In [14]:
duplicate_list=[1,2,3]